In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-07-12 17:49:15--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.07s   

2025-07-12 17:49:16 (16.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [8]:
# Create a set of all the characters in this text - get a list of them and sort them to get them in correct ASCII order
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
# Tokenizing the text (develop a strategy) - Convert the raw text, convert them to a series/sequence according to some vocabulary
# So here it's a character level model - translating individual chars into integers.

stoi = { ch:i for i, ch in enumerate(chars) } # Note the syntax on how easy it is for mapping to a dictionary!
itos = { i:ch for i, ch in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: [''.join(itos[i] for i in l)]

print(encode('Hello Im learning'))
print(decode(encode('Hello Im learning')))

# SentencePiece by google - subword tokenizer (google)
# tiktoken (gpt)

[20, 43, 50, 50, 53, 1, 21, 51, 1, 50, 43, 39, 56, 52, 47, 52, 45]
['Hello Im learning']


In [18]:
# tiktoken
import tiktoken

print(tiktoken.list_encoding_names())
vocabulary = tiktoken.get_encoding('gpt2')

print(vocabulary.n_vocab)
print(vocabulary.encode('This is gpt2!'))


['gpt2', 'r50k_base', 'p50k_base', 'p50k_edit', 'cl100k_base', 'o200k_base']
50257
[1212, 318, 308, 457, 17, 0]


In [19]:
# We now encode the entire dataset of shakespeare into a tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])


/Users/gaurav/Desktop/NanoGPT/env/lib/python3.10/site-packages/torch/_subclasses/functional_tensor.py:276: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
# Now, an important step is to make sure we have our train and test data separate.
n = int(0.9 * len(data)) # 90:10 split
train_data = data[:n]
val_data = data[n:] # Where data is the encoded corpus

In [ ]:
# When we train a transformer, we sample chunks and train it on chunks at a time.
# They have a fixed maximum length
block_size = 8
train_data[:block_size + 1]
# When you sample a chunk, this has multiple examples packed. All chars follow each other.
# When you plug this in, you simultaneously try teaching the model to predict at each of these positions.
# So in a chunk of 9, you have 8 examples ((18, 47), (47, 56) ... ) where the second character follows the first.


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size + 1] # one offset from x to have the following characters.
# So this will mean, for x[character_index], it is followed by y[character_index]
for t in range(block_size):
    context = x[:t + 1]
    target = y[t]

    print(f"{context} is the context for {target}")

# This is basically if im not wrong, how autoregressively this works, where a future output is dependent on all the past inputs.
# We train on all these context sizes (1-8) because we want our network get used to seeing contexts, all the way from 1 to block-size (everything in between).
# We can sample with as little as 1 context, and it predicts up to block size - after which we truncate (from the front - old things basically) - since the transformer can only take a max of <batch_size> inputs.

tensor([18]) is the context for 47
tensor([18, 47]) is the context for 56
tensor([18, 47, 56]) is the context for 57
tensor([18, 47, 56, 57]) is the context for 58
tensor([18, 47, 56, 57, 58]) is the context for 1
tensor([18, 47, 56, 57, 58,  1]) is the context for 15
tensor([18, 47, 56, 57, 58,  1, 15]) is the context for 47
tensor([18, 47, 56, 57, 58,  1, 15, 47]) is the context for 58


In [ ]:
# The above takes into account the TIME dimension.
# Now, to deal with BATCH dimension. We feed multiple batches - all stacked up
# GPUs are very good at parallel processing, so we want to keep the GPU efficient and busy.
torch.manual_seed(1337) # Reproducibility
batch_size = 4 # Number of independent sequences that we can parallelly process.
block_size = 8 # Maximum context length for a given prediction

def get_batch(split):
    data = train_data if split == 'train' else val_data
    # Random sampling points in our dataset
    ix = torch.randint(len(data) - block_size, (batch_size,)) # (high, (size)) where the size is a tuple defining the shape of the output. This simply means, since batch_size = 4, we get 4 values randomly generated in the range (0, len(data) - block_size)
    # Our first block_size characters (context)
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix]) # Stacks them as rows (where each row is a block of <block_size> sampled from index <i>)
    # Our offset characters (targets)
    y = torch.stack([data[i+1 : i+block_size+1] for i in ix])
    # This will help us create our loss function and give us the correct answer for every single position
    return x, y

xb, yb = get_batch('train')
print('inputs: ')
print(xb.shape)
print(xb)
print('targets: ')
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1] # All characters of that batch, preceeding our target
        target = yb[b, t] # Note that the yb characters are all offset to the right by 1
        print(f'when the input is {context}, the output should be {target}')

# Note that each and every single one of the rows in xb or yb, are completely independent

   

tensor([ 76049, 234249, 934904, 560986])
inputs: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when the input is tensor([24]), the output should be 43
when the input is tensor([24, 43]), the output should be 58
when the input is tensor([24, 43, 58]), the output should be 5
when the input is tensor([24, 43, 58,  5]), the output should be 57
when the input is tensor([24, 43, 58,  5, 57]), the output should be 1
when the input is tensor([24, 43, 58,  5, 57,  1]), the output should be 46
when the input is tensor([24, 43, 58,  5, 57,  1, 46]), the output should be 43
when the input is tensor([24, 43, 58,  5, 57,  1, 46, 43]), the output should be 39
when t

In [ ]:
# Now, know that xb is our BATCH of input that we are going to feed into the transformer
# Start with the simplest possible neural net for a language model - bigram model.